In [ ]:
!pip install dash

In [4]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [12]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")

app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
        html.Div([
            #drop down INPUT to select the yearly or recession period statistics
            html.Div([
                dcc.Dropdown(
                    id = 'dropdown-statistics',
                    options = [
                        {'label':'Yearly Statistics', 'value':'Yearly Statistics'},
                        {'label':'Recession Period Statistics', 'value':'Recession Period Statistics'}
                    ],
                    placeholder = 'Select a Report Type: ',
                    value = 'Select Statistics',
                    style = {'textAlign':'center', 'font-size':20}
                  )
            ]),
            #drop down INPUT to select the year
            html.Div([
                dcc.Dropdown(
                    id = 'select-year',
                    options = [{'label': i, 'value': i} for i in range(1980,2024,1)],
                    placeholder = 'Select-year',
                    value = 'Select-year'
                )
            ]),

            #OUTPUT
            html.Div([
                html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
            ])
      ])

  ]
)

@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)


def update_input_container(selected_statistics):
    if selected_statistics =='Yearly Statistics':
        return False
    else:
        return True

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics',component_property='value'), #INPUT report type
    Input(component_id='select-year',component_property='value')] #INPUT year selection

    )

def update_output_container(selected_statistics, input_year):
  #1st case, if the choosen input is recession period
    if (selected_statistics == 'Recession Period Statistics'):
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]

        #plot 1
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                x='Year',
                y='Automobile Sales',
                title="Automobile Sales fluctuate over Recession Period"))

        #plot 2
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle Type',
            y='Automobile Sales',
            title="The Average of Vehicles Sold by Its Type"))

        #plot 3
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
                figure=px.pie(exp_rec,
                values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Total Expenditure Share by Vehicle Type"))

        #plot 4
        unemp_data= recession_data.groupby(['Vehicle_Type', 'Automobile_Sales'])['unemployment_rate'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data,
        x = 'unemployment_rate',
        y = 'Automobile_Sales',
        color='Vehicle_Type',
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales'))

        return [html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})]

  #2nd case, if the choosen input is yearly statistics
    elif (input_year and selected_statistics == 'Yearly Statistics'):
        data = df[df['Year'] == input_year]

        #plot 1
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas,
                           x = 'Year',
                           y = 'Automobile_Sales',
                           title = 'Yearly Automobile Sales')
            )

        #plot 2
        mas=data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas,
            x='Month',
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))

        #plot 3
        avr_vdata = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure= px.bar(avr_vdata,
            x = 'Year',
            y = 'Automobile_Sales',
            title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

        #plot 4
        exp_data = data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title='Total Advertisment Expenditure for Each Vehicle'))

        return [html.Div(className='chart-item', children=[html.Div(Y_chart1),html.Div(Y_chart2)],style={'display':'flex'}),
                html.Div(className='chart-item', children=[html.Div(Y_chart3),html.Div(Y_chart4)],style={'display': 'flex'})]

    else:
      return None

if __name__ == '__main__':
  app.run_server(debug = True)


<IPython.core.display.Javascript object>

In [13]:
#YA INI PNYA RICAT YGY
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years
year_list = [i for i in range(1980, 2024, 1)]
#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center', 'color': '#503D36',
                                                'font-size': 24}),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options = dropdown_options,
            value='Select Statistics',
            placeholder='Select a report type',
            style={'width':'80%',
            'padding':'3px',
            'font-size': '20px',
            'text-align-last': 'center'}
        )
    ]),
    html.Div([
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list]
        )
    ]),
    html.Div([#TASK 2.3: Add a division for output display
        html.Div(id='output-container', className=' chart-grid', style={'display':'flex'}),
    ])
])
#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))

def update_input_container(selected_statistics):
    if selected_statistics =='Yearly Statistics':
        return False
    else:
        return True

#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
    Input(component_id='dropdown-statistics', component_property='value')])


def update_output_container(selected_year, selected_statistics):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        #Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        # use groupby to create relevant data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period"))

        #Plot 2 Calculate the average number of vehicles sold by vehicle type
        # use groupby to create relevant data for plotting
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title="Average Vehicle Sales by Type during Recession"))

        # Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        # use groupby to create relevant data for plotting
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title="Expenditure Share by Vehicle Type during Recession"))

        # Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        unemployment_rate = recession_data.groupby('Vehicle_Type')['unemployment_rate'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemployment_rate,
            x='Vehicle_Type',
            y='unemployment_rate',
            title="Effect of Unemployment Rate on Vehicle Type and Sales during Recession"))

        return [
            html.Div(className='chart-item', children=[html.Div(R_chart1), html.Div(R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(R_chart3), html.Div(R_chart4)], style={'display': 'flex'})
        ]

    # TASK 2.6: Create and display graphs for Yearly Report Statistics
    elif (selected_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == selected_year]

        #plot 1 Yearly Automobile sales using line chart for the whole period.
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas,
            x='Year',
            y='Automobile_Sales',
            title='Yearly Automobile Sales'))

        # Plot 2 Total Monthly Automobile sales using line chart.
        total_monthly_sales = yearly_data.groupby(['Year', 'Month'])['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(total_monthly_sales,
            x='Month',
            y='Automobile_Sales',
            title='Total Monthly Automobile Sales'))

        # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata,
            x='Vehicle_Type',
            y='Automobile_Sales',
            title=f'Average Vehicles Sold by Vehicle Type in the year {selected_year}'))

        # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
            values='Advertising_Expenditure',
            names='Vehicle_Type',
            title='Total Advertisement Expenditure for each vehicle'))

        return [
            html.Div(className='chart-item', children=[html.Div(Y_chart1), html.Div(Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(Y_chart3), html.Div(Y_chart4)], style={'display': 'flex'})
        ]

    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>